In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2021, day=24)

def parses(text):
    program = [line.split() for line in text.strip().split('\n')]
    program = [(instr, tuple([r if r in 'wxyz' else int(r) for r in args])) for instr, *args in program]
    return program

data = parses(puzzle.input_data)

In [25]:
import operator

In [14]:
operator.mod

<function _operator.mod(a, b, /)>

In [2]:
import operator
def op_builder(binop):
    def op(regs, dst, arg):
        regs[dst] = binop(regs[dst], arg if isinstance(arg, int) else regs[arg])
    return op
    
ops = {
    'add': op_builder(operator.add),
    'sub': op_builder(operator.sub),
    'mul': op_builder(operator.mul),
    'div': op_builder(operator.floordiv),
    'mod': op_builder(operator.mod),
    'eql': op_builder(lambda a, b: int(a == b)),
}

def simulate(program, inputs):
    steps = len(inputs)
    i = 0
    regs = defaultdict(int)
    for instr, args in program:
        if instr == 'inp':
            if i >= steps:
                return regs
            regs[args[0]] = inputs[i]
            i += 1
        else:
#             print(instr, args)
            a, b = args
            val = b if isinstance(b, int) else regs[b]
            if instr == 'div':
                assert val != 0
            if instr == 'mod':
                assert regs[a] >= 0 and val > 0
            ops[instr](regs, a, b)
    return regs

In [3]:
simulate(data, [9])

defaultdict(int, {'w': 9, 'x': 1, 'z': 15, 'y': 15})

In [337]:
program = data

In [338]:
steps = [program[18*i:18*(i+1)] for i in range(14)]

vals = [tuple(step[i][1][1] for i in [4,5,15]) for step in steps]

In [381]:
[v[0] f]

[(1, 11, 6),
 (1, 13, 14),
 (1, 15, 14),
 (26, -8, 10),
 (1, 13, 9),
 (1, 15, 12),
 (26, -11, 8),
 (26, -4, 13),
 (26, -15, 12),
 (1, 14, 6),
 (1, 14, 9),
 (26, -1, 15),
 (26, -8, 4),
 (26, -14, 10)]

In [379]:
simulate(data, [9, 9])

defaultdict(int, {'w': 9, 'x': 1, 'z': 413, 'y': 23})

In [446]:
def run_step(z, w, i):
    A, B, C = vals[i]
    x = int((z % 26) != w - B)
    z //= A  ## A either 1 or 26
    z *= 25 * x + 1
    z += (w + C) * x
    return z

In [447]:
# for i in range(1,15):
#     print(simulate(data, ns[:i])['z'])

In [454]:
len(steps)

14

In [464]:
def solve(program, part):
    steps = [program[18*i:18*(i+1)] for i in range(14)]
    vals = [tuple(step[i][1][1] for i in [4,5,15]) for step in steps]
    
    def run_step(z, w, i):
        A, B, C = vals[i]
        x = int((z % 26) != w - B)
        z //= A  ## A either 1 or 26
        z *= 25 * x + 1
        z += (w + C) * x
        return z, x

    stack = [(0,0, tuple())]
    
    while True:
        z, i, seq = stack.pop()
        for w in (range(1,10) if part == 'a' else range(9,0,-1)):
            z2, x = run_step(z, w, i)

            if vals[i][0] == 26 and x != 0:
                continue
            seq2 = seq + (w,)
            if z2 == 0 and i == 13:
                return int("".join(map(str,seq2)))
            elif i < 13:
                stack.append((z2, i+1, seq2))

In [465]:
dfs(data, 'a')

TypeError: dfs() missing 1 required positional argument: 'part'

In [466]:
dfs(data, 'b')

92171126131911

In [448]:
z = 0
for i in range(14):
    z = run_step(z, ns[i], i)
#     print(z)

1 0 11
1 15 13
1 23 15
26 15 -8
1 23 13
1 11 15
26 14 -11
26 11 -4
26 23 -15
1 15 14
1 15 14
26 10 -1
26 15 -8
26 15 -14


In [453]:
z = 0
for i, n in enumerate([9, 9, 3, 9]):
    z = run_step(z, n, i)
    print(z)

1 0 11
15
1 15 13
413
1 23 15
10755
26 17 -8
413


In [435]:
vals

[(1, 11, 6),
 (1, 13, 14),
 (1, 15, 14),
 (26, -8, 10),
 (1, 13, 9),
 (1, 15, 12),
 (26, -11, 8),
 (26, -4, 13),
 (26, -15, 12),
 (1, 14, 6),
 (1, 14, 9),
 (26, -1, 15),
 (26, -8, 4),
 (26, -14, 10)]

In [436]:
vals[3]

(26, -8, 10)

In [432]:
for i in range(1,10):
    print(run_step(10755, i, 3))

10749
10750
10751
10752
10753
10754
10755
10756
413


In [400]:
ns

[9, 9, 1, 7, 2, 2, 3, 7, 8, 9, 1, 9, 7, 1]

In [433]:
for i in range(1,10):
    print(run_step(0, i, 0))

7
8
9
10
11
12
13
14
15


In [434]:
for z in range(7,16):
    for i in range(1,10):
        print(run_step(z, i, 1), end='\t')
    print('')

197	198	199	200	201	202	203	204	205	
223	224	225	226	227	228	229	230	231	
249	250	251	252	253	254	255	256	257	
275	276	277	278	279	280	281	282	283	
301	302	303	304	305	306	307	308	309	
327	328	329	330	331	332	333	334	335	
353	354	355	356	357	358	359	360	361	
379	380	381	382	383	384	385	386	387	
405	406	407	408	409	410	411	412	413	


In [274]:
simulate(data, [9, 9, 1, 7, 2, 2, 3, 7, 8, 9, 1, 9, 7, 1])

defaultdict(int, {'w': 1, 'x': 0, 'z': 0, 'y': 0})

In [269]:
len([9, 9, 1, 7, 2, 2, 3, 7, 8, 9, 1, 9, 7, 1])

14

In [209]:
ns

[9, 9, 1, 7, 2, 2, 3, 7, 8, 9, 1, 9, 7, 1]

In [495]:
from z3 import Int, If, Optimize, Sum, Solver, ArithRef, IntNumRef, simplify

In [494]:
type(simplify(If(x>0,1,-1)*0))

z3.z3.IntNumRef

In [487]:
ArithRef.__floordiv__ = ArithRef.__truediv__

In [488]:
import operator
def op_builder(binop):
    def op(regs, dst, arg):
        return binop(regs[dst], arg if isinstance(arg, int) else regs[arg])
    return op
    
ops = {
    'add': op_builder(operator.add),
    'sub': op_builder(operator.sub),
    'mul': op_builder(operator.mul),
    'div': op_builder(operator.floordiv),
    'mod': op_builder(operator.mod),
    'eql': op_builder(lambda a, b: If(a==b, 1, 0)),
}

In [195]:
def solve(program):
    opt = Solver()
    i = 0
    regs = {k: 0 for k in 'wxyz'}

    inputs = [Int(f'inp{i:02d}') for i in range(14)]

    for inp in inputs:
        opt.add( 1 <= inp)
        opt.add( inp <= 9)
    
    for j, (instr, args) in enumerate(program):
        if instr == 'inp':
            regs[args[0]] = inputs[i]
            i += 1
        else:
            a, b = args                
            if instr == 'div':
                if not isinstance(b, int):
                    opt.add(regs[b] != 0)
            if instr == 'mod':
                opt.add(regs[a] >= 0)
                if not isinstance(b, int):
                    opt.add(regs[b] > 0)
            if instr == 'mul' and args[1] == 0:
                regs[a] = 0
            else:
                result = Int(f'R{j:03d}')
                opt.add(result == ops[instr](regs, a, b))
                regs[a] = result
            
    opt.add(regs['z'] == 0)
    return opt, inputs

In [196]:
opt, inputs = solve(data)

In [197]:
opt.check()

sat

In [199]:
solution = opt.model()

In [200]:
ns = [opt.model()[inp].as_long() for inp in inputs]

In [201]:
int("".join(str(d) for d in ns))

99172237891971

In [172]:
2+2

4

In [160]:
sorted(str(a) for a in opt.model())

['R001',
 'R002',
 'R003',
 'R004',
 'R005',
 'R006',
 'R007',
 'R008',
 'R009',
 'R010',
 'R011',
 'R012',
 'R013',
 'R014',
 'R015',
 'R016',
 'R017',
 'R019',
 'R020',
 'R021',
 'R022',
 'R023',
 'R024',
 'R025',
 'R026',
 'R027',
 'R028',
 'R029',
 'R030',
 'R031',
 'R032',
 'R033',
 'R034',
 'R035',
 'R037',
 'R038',
 'R039',
 'R040',
 'R041',
 'R042',
 'R043',
 'R044',
 'R045',
 'R046',
 'R047',
 'R048',
 'R049',
 'R050',
 'R051',
 'R052',
 'R053',
 'R055',
 'R056',
 'R057',
 'R058',
 'R059',
 'R060',
 'R061',
 'R062',
 'R063',
 'R064',
 'R065',
 'R066',
 'R067',
 'R068',
 'R069',
 'R070',
 'R071',
 'R073',
 'R074',
 'R075',
 'R076',
 'R077',
 'R078',
 'R079',
 'R080',
 'R081',
 'R082',
 'R083',
 'R084',
 'R085',
 'R086',
 'R087',
 'R088',
 'R089',
 'R091',
 'R092',
 'R093',
 'R094',
 'R095',
 'R096',
 'R097',
 'R098',
 'R099',
 'R100',
 'R101',
 'R102',
 'R103',
 'R104',
 'R105',
 'R106',
 'R107',
 'R109',
 'R110',
 'R111',
 'R112',
 'R113',
 'R114',
 'R115',
 'R116',
 'R117',
 

In [108]:
opt.check()

unknown

In [102]:
O.check()

unknown

In [ ]:
from z3 import Int, If, Optimize, Sum, Solver, ArithRef, IntNumRef, simplify

In [535]:
def symbolic_exec(program, inputs=[]):

    s = Solver()
    i = 0
    regs = {k: 0 for k in 'wxyz'}

    inputs = inputs + [Int(f'inp{i:02d}') for i in range(14-len(inputs))]

    for inp in inputs:
        s.add( 1 <= inp)
        s.add( inp <= 9)
    
    for j, (instr, args) in enumerate(program):
        if instr == 'inp':
            regs[args[0]] = inputs[i]
            i += 1
        else:
            a, b = args                
            val = ops[instr](regs, a, b)
            if not isinstance(val, int):
                val = simplify(val)
                if isinstance(val, IntNumRef):
                    val = val.as_long()
                    
            if not isinstance(val, int):
                result = Int(f'R{j:03d}')
                s.add(result == val)
                regs[a] = result
            else:
                regs[a] = val

    s.add(regs['z'] == 0)    
    return str(s.check()) == 'sat'

In [537]:
def solve_a(data):
    digits = []
    for i in tqdm(range(14)):
        for n in range(9,0,-1):
            if symbolic_exec(data, digits+[n]):
                print(n)
                digits.append(n)
                break
    return "".join(map(str,digits))

In [ ]:
solve_a(data)

  0%|          | 0/14 [00:00<?, ?it/s]

---

---

---

---